## STA 141B 
### Homework 3

### Your Name: 

Submit only this ipynb to canvas.  Work in between the exercise cells and be clear about your answers.  For example, you should add cells below the cell starting with 1.1 for the answer to 1.1.  You can break your answers into as many cells as you like.  Please do not clear your output so that we can see your answers without running all of the cells.

Throughout this homework we will be using the the covidcast api, and you can find [the documentation here](https://cmu-delphi.github.io/delphi-epidata/).  We will be using the http api and you should not use the `covidcast` python package.  Even though it is available, I want to test you on using the requests package and making http requests directly.

In [48]:
import requests
import pandas as pd
import io
import numpy as np
import sqlalchemy as sqla
import requests_cache
import plotnine as p9
from pandasql import sqldf
requests_cache.install_cache('covidcast_cache')

Read [the documentation about the Epidata API at this README](https://cmu-delphi.github.io/delphi-epidata/api/covidcast.html) and we will be using the covidcast "endpoint".  Pay special attention to the section "Constructing API Queries".  You will be making requests with the request package.  Consider the following get request...
```
https://api.covidcast.cmu.edu/epidata/api.php?endpoint=covidcast&data_source=fb-survey&signal=smoothed_cli&time_type=day&geo_type=county&time_values=20200406-20200410&geo_value=06001
```
If you put this in your browser you see...
```
{ "epidata": [{"geo_value":"06001","signal":"smoothed_cli","time_value":20200406,"direction":null,"issue":20200903,"lag":150,"value":0.0,"stderr":0.3691187,"sample_size":133.079},{"geo_value":"06001","signal":"smoothed_cli","time_value":20200407,"direction":null,"issue":20200903,"lag":149,"value":0.7510297,"stderr":0.3720838,"sample_size":422.8531},{"geo_value":"06001","signal":"smoothed_cli","time_value":20200408,"direction":null,"issue":20200903,"lag":148,"value":0.6378432,"stderr":0.2732424,"sample_size":685.658},{"geo_value":"06001","signal":"smoothed_cli","time_value":20200409,"direction":null,"issue":20200903,"lag":147,"value":0.6162563,"stderr":0.2433975,"sample_size":871.1857},{"geo_value":"06001","signal":"smoothed_cli","time_value":20200410,"direction":null,"issue":20200903,"lag":146,"value":0.7180705,"stderr":0.2483399,"sample_size":979.2376}], "result": 1, "message": "success" }
```
which is the json returned from this query.  The parameters for the query are 
```
endpoint=covidcast
data_source=fb-survey
signal=smoothed_cli
time_type=day
geo_type=county
time_values=20200406-20200410
geo_value=06001
```
which asks for the smoothed cli signal from the fb survey data for county 06001 from 2020-04-06 to 2020-04-10.  Notice that if you want to select all geo_values then you can set geo_value=*.

**Exercise 1.** Request the value for the smoothed_cli signal from the fb-survey datasource on 2020-10-01 for ALL STATES.  Notice that the above is for counties and so you need to change this to states.  Do the same for smoothed_hh_cmnty_cli signal from the fb-survey data, and the confirmed_7dav_incidence_prop signal from the jhu-csse data source.  Create dataframes from these and join these 3 variables based on the state into a single dataframe.  Drop rows with missingness (there should be only about 5 of them dropped this way, corresponding to the US territories).  Finally, compute and output the spearman correlation between all variables in this dataframe (there should be 51 rows in the new dataset). 

In [109]:
#Requesting for smooth_cli signal
params = {'endpoint': 'covidcast', 'data_source': 'fb-survey', 'signal': 'smoothed_cli', 'time_type': 'day', 'geo_type': 'state', 'time_values': '20201001', 'geo_value': '*'}
covid_url = 'https://api.covidcast.cmu.edu/epidata/api.php'
req = requests.get(covid_url, params = params)
req.url

'https://api.covidcast.cmu.edu/epidata/api.php?data_source=fb-survey&endpoint=covidcast&geo_type=state&geo_value=%2A&signal=smoothed_cli&time_type=day&time_values=20201001'

In [110]:
#Requesting for smoothed_hh_cmnty_cli signal
params2 = {'endpoint': 'covidcast', 'data_source': 'fb-survey', 'signal': 'smoothed_hh_cmnty_cli', 'time_type': 'day', 'geo_type': 'state', 'time_values': '20201001', 'geo_value': '*'}
covid_url2 = 'https://api.covidcast.cmu.edu/epidata/api.php'
req2 = requests.get(covid_url2, params = params2)
req2.url

'https://api.covidcast.cmu.edu/epidata/api.php?data_source=fb-survey&endpoint=covidcast&geo_type=state&geo_value=%2A&signal=smoothed_hh_cmnty_cli&time_type=day&time_values=20201001'

In [111]:
#Requesting for smoothed_hh_cmnty_cli signal
params3 = {'endpoint': 'covidcast', 'data_source': 'jhu-csse', 'signal': 'confirmed_7dav_incidence_prop', 'time_type': 'day', 'geo_type': 'state', 'time_values': '20201001', 'geo_value': '*'}
covid_url3 = 'https://api.covidcast.cmu.edu/epidata/api.php'
req3 = requests.get(covid_url3, params = params3)
req3.url

'https://api.covidcast.cmu.edu/epidata/api.php?data_source=jhu-csse&endpoint=covidcast&geo_type=state&geo_value=%2A&signal=confirmed_7dav_incidence_prop&time_type=day&time_values=20201001'

In [112]:
#Json for 1st request
dd = req.json()
dd

{'epidata': [{'geo_value': 'ak',
   'signal': 'smoothed_cli',
   'time_value': 20201001,
   'direction': None,
   'issue': 20201119,
   'lag': 49,
   'value': 1.5854157,
   'stderr': 0.4292574,
   'sample_size': 760.0},
  {'geo_value': 'al',
   'signal': 'smoothed_cli',
   'time_value': 20201001,
   'direction': None,
   'issue': 20201119,
   'lag': 49,
   'value': 0.9832923,
   'stderr': 0.1406174,
   'sample_size': 3742.0925},
  {'geo_value': 'ar',
   'signal': 'smoothed_cli',
   'time_value': 20201001,
   'direction': None,
   'issue': 20201119,
   'lag': 49,
   'value': 1.0576546,
   'stderr': 0.1735594,
   'sample_size': 2586.9222},
  {'geo_value': 'az',
   'signal': 'smoothed_cli',
   'time_value': 20201001,
   'direction': None,
   'issue': 20201119,
   'lag': 49,
   'value': 0.5973764,
   'stderr': 0.0904563,
   'sample_size': 5681.9867},
  {'geo_value': 'ca',
   'signal': 'smoothed_cli',
   'time_value': 20201001,
   'direction': None,
   'issue': 20201119,
   'lag': 49,
   'v

In [113]:
#1st data frame
df = pd.DataFrame(dd['epidata'])
df

,geo_value,signal,time_value,direction,issue,lag,value,stderr,sample_size
0,ak,smoothed_cli,20201001,None,20201119,49,1.585416,0.429257,760.0000
1,al,smoothed_cli,20201001,None,20201119,49,0.983292,0.140617,3742.0925
2,ar,smoothed_cli,20201001,None,20201119,49,1.057655,0.173559,2586.9222
3,az,smoothed_cli,20201001,None,20201119,49,0.597376,0.090456,5681.9867
4,ca,smoothed_cli,20201001,None,20201119,49,0.450001,0.039909,21930.1313
5,co,smoothed_cli,20201001,None,20201119,49,0.561476,0.091652,5137.0121
6,ct,smoothed_cli,20201001,None,20201119,49,0.444473,0.097750,3866.0000
7,dc,smoothed_cli,20201001,None,20201119,49,0.268025,0.223716,533.0000
8,de,smoothed_cli,20201001,None,20201119,49,0.202961,0.133807,1129.0146
9,fl,smoothed_cli,20201001,None,20201119,49,0.515880,0.047152,17767.0001


In [114]:
#2nd json
cc = req2.json()
cc

{'epidata': [{'geo_value': 'ak',
   'signal': 'smoothed_hh_cmnty_cli',
   'time_value': 20201001,
   'direction': None,
   'issue': 20201119,
   'lag': 49,
   'value': 24.6185853,
   'stderr': 1.6054524,
   'sample_size': 720.0},
  {'geo_value': 'al',
   'signal': 'smoothed_hh_cmnty_cli',
   'time_value': 20201001,
   'direction': None,
   'issue': 20201119,
   'lag': 49,
   'value': 25.3939814,
   'stderr': 0.7367741,
   'sample_size': 3490.0847},
  {'geo_value': 'ar',
   'signal': 'smoothed_hh_cmnty_cli',
   'time_value': 20201001,
   'direction': None,
   'issue': 20201119,
   'lag': 49,
   'value': 29.2446711,
   'stderr': 0.9270044,
   'sample_size': 2407.9216},
  {'geo_value': 'az',
   'signal': 'smoothed_hh_cmnty_cli',
   'time_value': 20201001,
   'direction': None,
   'issue': 20201119,
   'lag': 49,
   'value': 16.5858781,
   'stderr': 0.5083493,
   'sample_size': 5353.6945},
  {'geo_value': 'ca',
   'signal': 'smoothed_hh_cmnty_cli',
   'time_value': 20201001,
   'direction'

In [115]:
#2nd dataframe
df2 = pd.DataFrame(cc['epidata'])
df2

,geo_value,signal,time_value,direction,issue,lag,value,stderr,sample_size
0,ak,smoothed_hh_cmnty_cli,20201001,None,20201119,49,24.618585,1.605452,720.0000
1,al,smoothed_hh_cmnty_cli,20201001,None,20201119,49,25.393981,0.736774,3490.0847
2,ar,smoothed_hh_cmnty_cli,20201001,None,20201119,49,29.244671,0.927004,2407.9216
3,az,smoothed_hh_cmnty_cli,20201001,None,20201119,49,16.585878,0.508349,5353.6945
4,ca,smoothed_hh_cmnty_cli,20201001,None,20201119,49,13.689740,0.242334,20120.1215
5,co,smoothed_hh_cmnty_cli,20201001,None,20201119,49,17.172828,0.539824,4881.0126
6,ct,smoothed_hh_cmnty_cli,20201001,None,20201119,49,12.582690,0.550696,3627.0000
7,dc,smoothed_hh_cmnty_cli,20201001,None,20201119,49,11.100000,1.406249,499.0000
8,de,smoothed_hh_cmnty_cli,20201001,None,20201119,49,13.781560,1.057752,1062.0146
9,fl,smoothed_hh_cmnty_cli,20201001,None,20201119,49,18.271586,0.302169,16355.0001


In [116]:
#3rd json
aa = req3.json()
aa

{'epidata': [{'geo_value': 'ak',
   'signal': 'confirmed_7dav_incidence_prop',
   'time_value': 20201001,
   'direction': None,
   'issue': 20210403,
   'lag': 184,
   'value': 16.5329512,
   'stderr': None,
   'sample_size': None},
  {'geo_value': 'al',
   'signal': 'confirmed_7dav_incidence_prop',
   'time_value': 20201001,
   'direction': None,
   'issue': 20201119,
   'lag': 49,
   'value': 18.83323944392413,
   'stderr': None,
   'sample_size': None},
  {'geo_value': 'ar',
   'signal': 'confirmed_7dav_incidence_prop',
   'time_value': 20201001,
   'direction': None,
   'issue': 20201030,
   'lag': 29,
   'value': 27.323558076383637,
   'stderr': None,
   'sample_size': None},
  {'geo_value': 'as',
   'signal': 'confirmed_7dav_incidence_prop',
   'time_value': 20201001,
   'direction': None,
   'issue': 20210219,
   'lag': 141,
   'value': 0.0,
   'stderr': None,
   'sample_size': None},
  {'geo_value': 'az',
   'signal': 'confirmed_7dav_incidence_prop',
   'time_value': 20201001,


In [117]:
#3rd dataframe
df3 = pd.DataFrame(aa['epidata'])
df3

,geo_value,signal,time_value,direction,issue,lag,value,stderr,sample_size
0,ak,confirmed_7dav_incidence_prop,20201001,None,20210403,184,16.532951,None,None
1,al,confirmed_7dav_incidence_prop,20201001,None,20201119,49,18.833239,None,None
2,ar,confirmed_7dav_incidence_prop,20201001,None,20201030,29,27.323558,None,None
3,as,confirmed_7dav_incidence_prop,20201001,None,20210219,141,0.000000,None,None
4,az,confirmed_7dav_incidence_prop,20201001,None,20201030,29,6.594569,None,None
5,ca,confirmed_7dav_incidence_prop,20201001,None,20210414,195,8.300867,None,None
6,co,confirmed_7dav_incidence_prop,20201001,None,20201002,1,9.925293,None,None
7,ct,confirmed_7dav_incidence_prop,20201001,None,20201030,29,5.088751,None,None
8,dc,confirmed_7dav_incidence_prop,20201001,None,20201030,29,5.100964,None,None
9,de,confirmed_7dav_incidence_prop,20201001,None,20201030,29,12.323314,None,None


In [118]:
#1st merge
merged_df = pd.merge(df, df2, on="geo_value")
merged_df

,geo_value,signal_x,time_value_x,direction_x,issue_x,lag_x,value_x,stderr_x,sample_size_x,signal_y,time_value_y,direction_y,issue_y,lag_y,value_y,stderr_y,sample_size_y
0,ak,smoothed_cli,20201001,None,20201119,49,1.585416,0.429257,760.0000,smoothed_hh_cmnty_cli,20201001,None,20201119,49,24.618585,1.605452,720.0000
1,al,smoothed_cli,20201001,None,20201119,49,0.983292,0.140617,3742.0925,smoothed_hh_cmnty_cli,20201001,None,20201119,49,25.393981,0.736774,3490.0847
2,ar,smoothed_cli,20201001,None,20201119,49,1.057655,0.173559,2586.9222,smoothed_hh_cmnty_cli,20201001,None,20201119,49,29.244671,0.927004,2407.9216
3,az,smoothed_cli,20201001,None,20201119,49,0.597376,0.090456,5681.9867,smoothed_hh_cmnty_cli,20201001,None,20201119,49,16.585878,0.508349,5353.6945
4,ca,smoothed_cli,20201001,None,20201119,49,0.450001,0.039909,21930.1313,smoothed_hh_cmnty_cli,20201001,None,20201119,49,13.689740,0.242334,20120.1215
5,co,smoothed_cli,20201001,None,20201119,49,0.561476,0.091652,5137.0121,smoothed_hh_cmnty_cli,20201001,None,20201119,49,17.172828,0.539824,4881.0126
6,ct,smoothed_cli,20201001,None,20201119,49,0.444473,0.097750,3866.0000,smoothed_hh_cmnty_cli,20201001,None,20201119,49,12.582690,0.550696,3627.0000
7,dc,smoothed_cli,20201001,None,20201119,49,0.268025,0.223716,533.0000,smoothed_hh_cmnty_cli,20201001,None,20201119,49,11.100000,1.406249,499.0000
8,de,smoothed_cli,20201001,None,20201119,49,0.202961,0.133807,1129.0146,smoothed_hh_cmnty_cli,20201001,None,20201119,49,13.781560,1.057752,1062.0146
9,fl,smoothed_cli,20201001,None,20201119,49,0.515880,0.047152,17767.0001,smoothed_hh_cmnty_cli,20201001,None,20201119,49,18.271586,0.302169,16355.0001


In [119]:
#final merge
df_s = pd.merge(merged_df, df3, on="geo_value")
df_s

,geo_value,signal_x,time_value_x,direction_x,issue_x,lag_x,value_x,stderr_x,sample_size_x,signal_y,...,stderr_y,sample_size_y,signal,time_value,direction,issue,lag,value,stderr,sample_size
0,ak,smoothed_cli,20201001,None,20201119,49,1.585416,0.429257,760.0000,smoothed_hh_cmnty_cli,...,1.605452,720.0000,confirmed_7dav_incidence_prop,20201001,None,20210403,184,16.532951,None,None
1,al,smoothed_cli,20201001,None,20201119,49,0.983292,0.140617,3742.0925,smoothed_hh_cmnty_cli,...,0.736774,3490.0847,confirmed_7dav_incidence_prop,20201001,None,20201119,49,18.833239,None,None
2,ar,smoothed_cli,20201001,None,20201119,49,1.057655,0.173559,2586.9222,smoothed_hh_cmnty_cli,...,0.927004,2407.9216,confirmed_7dav_incidence_prop,20201001,None,20201030,29,27.323558,None,None
3,az,smoothed_cli,20201001,None,20201119,49,0.597376,0.090456,5681.9867,smoothed_hh_cmnty_cli,...,0.508349,5353.6945,confirmed_7dav_incidence_prop,20201001,None,20201030,29,6.594569,None,None
4,ca,smoothed_cli,20201001,None,20201119,49,0.450001,0.039909,21930.1313,smoothed_hh_cmnty_cli,...,0.242334,20120.1215,confirmed_7dav_incidence_prop,20201001,None,20210414,195,8.300867,None,None
5,co,smoothed_cli,20201001,None,20201119,49,0.561476,0.091652,5137.0121,smoothed_hh_cmnty_cli,...,0.539824,4881.0126,confirmed_7dav_incidence_prop,20201001,None,20201002,1,9.925293,None,None
6,ct,smoothed_cli,20201001,None,20201119,49,0.444473,0.097750,3866.0000,smoothed_hh_cmnty_cli,...,0.550696,3627.0000,confirmed_7dav_incidence_prop,20201001,None,20201030,29,5.088751,None,None
7,dc,smoothed_cli,20201001,None,20201119,49,0.268025,0.223716,533.0000,smoothed_hh_cmnty_cli,...,1.406249,499.0000,confirmed_7dav_incidence_prop,20201001,None,20201030,29,5.100964,None,None
8,de,smoothed_cli,20201001,None,20201119,49,0.202961,0.133807,1129.0146,smoothed_hh_cmnty_cli,...,1.057752,1062.0146,confirmed_7dav_incidence_prop,20201001,None,20201030,29,12.323314,None,None
9,fl,smoothed_cli,20201001,None,20201119,49,0.515880,0.047152,17767.0001,smoothed_hh_cmnty_cli,...,0.302169,16355.0001,confirmed_7dav_incidence_prop,20201001,None,20201030,29,10.711424,None,None


In [100]:
#Checking NaN's
df_s.isna()

,geo_value,signal_x,time_value_x,direction_x,issue_x,lag_x,value_x,stderr_x,sample_size_x,signal_y,...,stderr_y,sample_size_y,signal,time_value,direction,issue,lag,value,stderr,sample_size
0,False,False,False,True,False,False,False,False,False,False,...,False,False,False,False,True,False,False,False,True,True
1,False,False,False,True,False,False,False,False,False,False,...,False,False,False,False,True,False,False,False,True,True
2,False,False,False,True,False,False,False,False,False,False,...,False,False,False,False,True,False,False,False,True,True
3,False,False,False,True,False,False,False,False,False,False,...,False,False,False,False,True,False,False,False,True,True
4,False,False,False,True,False,False,False,False,False,False,...,False,False,False,False,True,False,False,False,True,True
5,False,False,False,True,False,False,False,False,False,False,...,False,False,False,False,True,False,False,False,True,True
6,False,False,False,True,False,False,False,False,False,False,...,False,False,False,False,True,False,False,False,True,True
7,False,False,False,True,False,False,False,False,False,False,...,False,False,False,False,True,False,False,False,True,True
8,False,False,False,True,False,False,False,False,False,False,...,False,False,False,False,True,False,False,False,True,True
9,False,False,False,True,False,False,False,False,False,False,...,False,False,False,False,True,False,False,False,True,True


In [101]:
#Calculating spearman coefficients
df_s.corr(method = "spearman")

,time_value_x,issue_x,lag_x,value_x,stderr_x,sample_size_x,time_value_y,issue_y,lag_y,value_y,stderr_y,sample_size_y,time_value,issue,lag,value
time_value_x,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
issue_x,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
lag_x,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
value_x,NaN,NaN,NaN,1.000000,0.598009,-0.284525,NaN,NaN,NaN,0.859276,0.460362,-0.282805,NaN,0.036695,0.036695,0.767421
stderr_x,NaN,NaN,NaN,0.598009,1.000000,-0.894208,NaN,NaN,NaN,0.425158,0.945068,-0.893394,NaN,-0.111484,-0.111484,0.411041
sample_size_x,NaN,NaN,NaN,-0.284525,-0.894208,1.000000,NaN,NaN,NaN,-0.141357,-0.963167,0.999367,NaN,0.159801,0.159801,-0.176199
time_value_y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
issue_y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
lag_y,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
value_y,NaN,NaN,NaN,0.859276,0.425158,-0.141357,NaN,NaN,NaN,1.000000,0.358371,-0.136923,NaN,-0.049657,-0.049657,0.935023


**Exercise 2.** Create and engine using sqlalchemy for a sqlite database (you can just use for example `covid.sqlite` as your location in your current directory).  You will sequentially pull these three variables above for each state at different dates  and write the resulting dataframes to a single table in the sqlite database.  Your date range should be from '2020-08-02' to '2021-04-04' at a frequency of every 7 days.  For each date, query the API for the three variables and construct the dataframe of each state at that date.  Then append this dataframe to a table (call the table "state") in the sqlite database that you created, using DataFrame.to_sql command.  Finally, make a sql query to this table that counts the number of rows in the state table, there should be 51 x the number of dates in your date range.

**Exercise 3.** Using `read_sql_query`, select all variables where the state is california from the state table, and read it into a pandas dataframe.  Filtering out CA should be done in SQL and then the pd.DataFrame should be constructed, the filter should not happen after you read in the full data.  In the end you should have one row for each date.  

For the two FB indicator variables, produce a plot comparing that to the death indicences.  Plot the FB indicator variables as a time series in a secondary y axis and the death indicence as the primary y axis.  You can just use pd.DataFrame.plot().  You should have the actual date on the X axis and not just an index number.  Remark on which indicator looks more variable and why they are good "leading indicators".  A leading indicator is a variable that rises and falls before the indicator of interest such as covid deaths.

In [49]:
pysqldf = lambda q: sqldf(q, globals())

In [102]:
list(df_s.columns)

['geo_value',
 'signal_x',
 'time_value_x',
 'direction_x',
 'issue_x',
 'lag_x',
 'value_x',
 'stderr_x',
 'sample_size_x',
 'signal_y',
 'time_value_y',
 'direction_y',
 'issue_y',
 'lag_y',
 'value_y',
 'stderr_y',
 'sample_size_y',
 'signal',
 'time_value',
 'direction',
 'issue',
 'lag',
 'value',
 'stderr',
 'sample_size']

In [103]:
q = """SELECT * 
       FROM df_s 
       WHERE geo_value = 'ca';"""
names = pysqldf(q)
names

,geo_value,signal_x,time_value_x,direction_x,issue_x,lag_x,value_x,stderr_x,sample_size_x,signal_y,...,stderr_y,sample_size_y,signal,time_value,direction,issue,lag,value,stderr,sample_size
0,ca,smoothed_cli,20201001,None,20201119,49,0.450001,0.039909,21930.1313,smoothed_hh_cmnty_cli,...,0.242334,20120.1215,confirmed_7dav_incidence_prop,20201001,None,20210414,195,8.300867,None,None


**Exercise 4.** Using read_sql_query, for each variable compute the average of all states grouped by date.  Read it into a pandas dataframe.  The averaging should be done in SQL and then the pd.DataFrame should be constructed, the average should not happen after you read in the full data. In the end you should have one row for each date.  The rest of this is identical to the previous exercise...

For the two FB indicator variables, produce a plot comparing that to the death indicences. Plot the FB indicator variables as a time series in a secondary y axis and the death indicence as the primary y axis. You can just use pd.DataFrame.plot(). You should have the actual date on the X axis and not just an index number.

In [107]:
solsqldf = lambda d: sqldf(d, globals())
d = """SELECT AVG(time_value_x), geo_value
       FROM df_s 
       GROUP BY time_value_x;"""
names2 = solsqldf(d)
names2

,AVG(time_value_x),geo_value
0,20201001.0,ak


**Exercise 5** The Pearson correlation between variables a and b can be computed from 5 quantities...
```
n = count of rows
cross = sum(a * b)
sqr_1 = sum(a * a)
sqr_2 = sum(b * b)
mu_1 = avg(a)
mu_2 = avg(b)
```
using the following equation
```
(cross - n*mu_1*mu_2) / ((sqr_1 - n*mu_1**2)**0.5 * (sqr_2 - n*mu_2**2)**0.5)
```
Using only SQL commands, compute these quantities for variables smoothed_hh_cmnty_cli and smoothed_cli and grouped by date, reading this into a DataFrame.  Compute the Pearson correlation for each date using these quantities, and plot the time series of pearson correlation. 

In [122]:
ddxsqldf = lambda n: sqldf(n, globals())
n = """SELECT COUNT(signal_y)/COUNT(signal_x), time_value_x
       FROM df_s
       GROUP BY time_value_x;"""
names3 = ddxsqldf(n)
names3

,COUNT(signal_y)/COUNT(signal_x),time_value_x
0,1,20201001
